In [0]:
# importing all the required header file
import pandas as pd 
import sqlite3
import os
import gc
import csv
from google.colab import drive
import numpy as np

In [2]:
# connecting data file from google
drive.mount('/content/gdrive')
# location of data
path='/content/gdrive/My Drive/intern_iitropar/Twitter-dataset/data/edges.csv'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
outputPath='/content/gdrive/My Drive/intern_iitropar/Processed Data of Twitter/'
inputPath='/content/gdrive/My Drive/intern_iitropar/Twitter-dataset/data/'

In [0]:
# coverting the required part of data to csv file and reallocating memory
def trimCSV(starting,Termination,name):
  if(os.path.exists('/content/gdrive/My Drive/intern_iitropar/Processed Data of Twitter/'+name+'.csv')==1):
                    os.remove('/content/gdrive/My Drive/intern_iitropar/Processed Data of Twitter/'+name+'.csv')
  df=pd.read_csv('/content/gdrive/My Drive/intern_iitropar/Twitter-dataset/data/edges.csv',names=['person','following'])
  if(not(starting==1)):
    df.drop(df[df.person < starting].index, inplace= True)
    df.drop(df[df.following < starting].index, inplace= True)
  df.drop(df[df.person > Termination].index, inplace=True)
  df.drop(df[df.following > Termination].index, inplace=True)
  df.set_index(['person','following'], inplace=True)
  df.to_csv('/content/gdrive/My Drive/intern_iitropar/Processed Data of Twitter/'+name+'.csv')
  del df
  gc.collect()
  df=pd.DataFrame()

In [0]:
trimCSV(1,2000,'edges')
print('completed Trim')
df=pd.read_csv('/content/gdrive/My Drive/intern_iitropar/Processed Data of Twitter/'+'edges.csv',names=['person','following'])
print('reading done')
con=sqlite3.connect('tempSql.sql')
df.to_sql('Data',con)
cur=con.cursor()
del df
gc.collect()
for p in range(1,2001):
  for f in range(p,2001):
    cur.execute('SELECT COUNT(*) from Data WHERE person=? and following=? and EXISTS (SELECT * from Data WHERE person=? and following=?)',(p,f,f,p))
    for x in cur:
      if(x==(0,)):
        cur.execute('DELETE from Data WHERE person=? and following=?',(f,p))
    con.commit()
print('SQL done')
query='SELECT * from Data '
data=pd.read_sql(query,con)
data.set_index('person',inplace=True)
data.to_csv('/content/gdrive/My Drive/intern_iitropar/Processed Data of Twitter/processedDataAfterProcessingPhase2.csv',columns=['following'])
cur.close()

completed Trim
reading done
SQL done


In [0]:
reqDataEndPoints=[200,400,600,800,1000,1200,1400,1600,1800,2000]
for size in reqDataEndPoints:
  df=pd.read_csv('/content/gdrive/My Drive/intern_iitropar//processedData.csv')
  con=sqlite3.connect(outputPath+'tempSQl'+str(size)+'.sql')
  df.to_sql('Data',con)
  cur=con.cursor()
  cur.execute('DELETE from Data where person>? or following>? or person<? or following<?',(size,size,size-199,size-199))
  con.commit()
  del df
  data=pd.read_sql('SELECT * from Data',con)
  data.set_index('person',inplace=True)
  data.to_csv(outputPath+'processedData'+str(size)+'.csv',columns=['following'])
  del data
  gc.collect()

In [0]:
# database creation
if (os.path.exists(outputPath+'DataBase.iitrpr')):
  con=sqlite3.connect(outputPath+'DataBase.iitrpr')
else:
  con=sqlite3.connect(outputPath+'DataBase.iitrpr')
  cur=con.cursor()
  cur.execute('CREATE TABLE Database (id INTEGER, following TEXT, origin TEXT, desitination TEXT)')

In [0]:
workingPath='/content/gdrive/My Drive/intern_iitropar/'
maximumPossibleUsers=11316811

In [0]:
# test passed!!! Hurray
if (os.path.exists(outputPath+'DataBase.iitrpr'))==1:
  con=sqlite3.connect(outputPath+'DataBase.iitrpr')
  cur=con.cursor()
else:
  con=sqlite3.connect(outputPath+'DataBase.iitrpr')
  cur=con.cursor()
  cur.execute('CREATE TABLE Database (id INTEGER, following TEXT, origin TEXT, destination TEXT)')
edges=pd.read_csv('/content/gdrive/My Drive/intern_iitropar/Processed Data of Twitter/AfterProcessing2.csv')
df=pd.read_csv(outputPath+'Selected4000LocationsOnMap.csv')
for person in range(0,2000):
  following=(edges.loc[edges['person']==person+1])['following'].tolist()
  weights=np.random.rand(len(following))*0.6+0.2
  following_weight=[]
  start=(df['lat1'][person],df['long1'][person])
  start=str(start)
  terminate=(df['lat2'][person],df['long2'][person])
  terminate=str(terminate)
  for x in range(len(following)):
    temp=(following[x]-1,weights[x])
    following_weight.append(temp)
  following_weight=str(following_weight)
  cur.execute('INSERT into Database (id,following,origin,destination) VALUES (?,?,?,?)',(person,following_weight,start,terminate))
  con.commit()
cur.execute('SELECT * from Database')
for x in cur:
	print(x)
cur.close()

(0, '[]', '(30.680220000000002, 76.73758000000001)', '(30.69022, 76.72758)')
(1, '[]', '(30.680220000000002, 76.73758000000001)', '(30.69022, 76.74758)')
(2, '[(5, 0.6183214455273618), (6, 0.3078402057662765), (15, 0.6494860070494457), (8, 0.6679914992030908), (9, 0.3155988956565439), (11, 0.7236412433302697), (10, 0.6123191643225345), (12, 0.6913776971466172), (16, 0.6473598331603713), (7, 0.397504491268439), (17, 0.4421589292836684), (13, 0.7805171429315538)]', '(30.680220000000002, 76.73758000000001)', '(30.70022, 76.72758)')
(3, '[(125, 0.46093010374464977), (1423, 0.37108819789070924), (1449, 0.7413675020344896), (1719, 0.6789686917428799)]', '(30.680220000000002, 76.73758000000001)', '(30.70022, 76.73758000000001)')
(4, '[(1744, 0.4111427926347686), (125, 0.35196112420839665), (408, 0.3140685431457082), (1449, 0.23234374132037017)]', '(30.680220000000002, 76.73758000000001)', '(30.70022, 76.76758000000001)')
(5, '[(2, 0.5027130746109904), (1744, 0.5938977035425241), (1791, 0.3109

In [0]:
# this part of code doesn't work
con=sqlite3.connect('Test.sql')
cur=con.cursor()
df=pd.read_csv('/content/gdrive/My Drive/intern_iitropar/Twitter-dataset/data/edges.csv',names=['person','following'])
# df=pd.read_csv('/content/gdrive/My Drive/intern_iitropar/Twitter-dataset/Processed Data of Twitter/edges.csv',names=['person','following'])
# df=pd.read_csv('/content/gdrive/My Drive/intern_iitropar/Processed Data of Twitter/'+'edges.csv',names=['person','following'])
print('reading done')
df.to_sql('Data',con)
del df
gc.collect()
df=pd.DataFrame()
con2=sqlite3.connect('tempDataBase.sql')
cur2=con2.cursor()
cur2.execute('CREATE TABLE tempDataBase (person INTEGER, following INTEGER)')
for p in range(1,maximumPossibleUsers+1):
  for f in range(p,maximumPossibleUsers+1):
    cur.execute('SELECT COUNT(*) from Data WHERE person=? and following=? and EXISTS (SELECT * from Data WHERE person=? and following=?)',(p,f,f,p))
    for x in cur:
      if(x==(0,)):
        cur.execute('DELETE from Data WHERE person=? and following=?',(f,p))
    con.commit()
    del cur
    cur=con.connect()
    gc.collect()
print('SQL DONE')
del p
del f
gc.collect()
for x in range(1,maximumPossibleUsers+1):
  cur.execute('SELECT person,following from Data WHERE person=?',(x,))
  for y in cur:
    cur2.execute('INSERT into tempDataBase (person,following) VALUES (?,?)',y)
    con2.commit()
cur2.execute('SELECT * from tempDataBase')
cur.close()
del cur
del con
gc.collect()
cur2.close()
data=pd.read_sql('SELECT * from tempDataBase',con2)
data.set_index('person',inplace=True)
data.to_csv(outputPath+'AfterProcessing2a.csv',columns=['following'])
del data
del cur2
del con2
gc.collect()
strg='sucess'
strg

reading done


In [0]:
# processing of google datafile
df=pd.read_csv('/content/gdrive/My Drive/intern_iitropar/chandigarh_road_data0.01 divisions.csv',names=['lat1','long1','lat2','long2','dis','time'])
df.drop(df[df.dis==0].index,inplace=True)
df=df.round(5)
df.to_csv(outputPath+'AfterRemoving0Maps.csv')
del df 
gc.collect()

431

In [0]:
df=pd.read_csv(outputPath+'AfterRemoving0Maps.csv',skiprows=1,names=['i','lat1','long1','lat2','long2','dis','time'])
# df.set_index('i',inplace=True)
x=np.random.permutation(np.arange(1,22199))[:4050]
df=df.loc[df.index.isin(x)]
df=df.head(4000)
df.drop('i',axis=1,inplace=True)
df=df.round(5)
df.set_index('lat1',inplace=True)
df.to_csv(outputPath+'Selected4000LocationsOnMap.csv')

In [22]:
df=pd.read_csv(outputPath+'Selected4000LocationsOnMap.csv')
df=df[:2000]
df.drop('dis',inplace=True,axis=1)
df.drop('time',inplace=True,axis=1)
print(df)
df=df.round(5)
df.set_index('lat1',inplace=True)
df.to_csv(outputPath+'PersonOriginAndSource.csv')

          lat1     long1      lat2     long2
0     30.68022  76.73758  30.69022  76.72758
1     30.68022  76.73758  30.69022  76.74758
2     30.68022  76.73758  30.70022  76.72758
3     30.68022  76.73758  30.70022  76.73758
4     30.68022  76.73758  30.70022  76.76758
5     30.68022  76.73758  30.70022  76.77758
6     30.68022  76.73758  30.71022  76.72758
7     30.68022  76.73758  30.71022  76.75758
8     30.68022  76.73758  30.72022  76.70758
9     30.68022  76.73758  30.72022  76.72758
10    30.68022  76.73758  30.72022  76.76758
11    30.68022  76.73758  30.72022  76.80758
12    30.68022  76.73758  30.73022  76.75758
13    30.68022  76.73758  30.74022  76.68758
14    30.68022  76.73758  30.74022  76.69758
15    30.68022  76.73758  30.74022  76.75758
16    30.68022  76.73758  30.74022  76.83758
17    30.68022  76.73758  30.75022  76.70758
18    30.68022  76.73758  30.75022  76.72758
19    30.68022  76.73758  30.75022  76.76758
20    30.68022  76.73758  30.75022  76.77758
21    30.6

In [42]:
df=pd.read_csv('/content/gdrive/My Drive/intern_iitropar/chandigarh_road_data0.01 divisions.csv',names=['lat1','long1','lat2','long2','dis','time'])
df=df.round(5)
y=df.loc[0,'dis']
print(np.asscalar(y))
df.set_index('lat1',inplace=True)
df.to_csv(outputPath+'distanceMatrix.csv')

0
